In [3]:
import cv2
import os
import numpy as np
import pandas as pd

In [4]:
import xlrd   #引入库
wb=xlrd.open_workbook(r"C:\Users\CANDY\Desktop\广告编码1-200.xlsx")
sheet_names=wb.sheet_names() 
sheet=wb.sheet_by_index(1)  
bvs=sheet.col_values(7)
titleyuanbens=sheet.col_values(2)
begins=sheet.col_values(3)
ends=sheet.col_values(4)

In [16]:
titleyuanbens

['视频标题',
 'Vlog｜一周生活 阶段性打工心得 出门前准备 打卡创意园',
 '我不想让别人夸我可爱，夸我身材好才是最安慰的！',
 '买这泥属于是为了这醋包的饺子了',
 '男生如何快速引起女生注意？',
 '我摆烂， 但赚了好几万。',
 '粉丝投稿：我失忆了，忘记了曾经相爱多年的男朋友',
 '无休妈妈喜获久违的独处时光。',
 '口臭算不算职场霸凌? 算不算家庭暴力? 赶紧漱口吧朋友们!',
 '谁还觉得电动牙刷是智商税啊? 看完这个实验立刻闭嘴惊艳!',
 '熬夜追剧不迷糊！！',
 '400块学的骑马，大伙看看值不值',
 '没有幸福？没关系，这次大象套套真的出手了！',
 '别太荒谬|私护得找专业的护理液',
 '除甲醛我吹爆！轻松跑出新房新家具的毒圈！',
 '在床上对女友说，你弱爆了？/关于女dom怎么教育女朋友',
 '趁姐姐睡着给她带上手铐...最后居然被反制？',
 '和女友同居两年，令我大哭特哭',
 '快乐最大化！',
 '30岁二胎宝妈医美初体验！肉眼可见变好看了？',
 '【助眠神器】枕头还能改善睡眠？别小看它，拯救失眠是关键！让你沾床就睡！',
 '女生一定要矜持？',
 '睡的着',
 '出发了从河南去新疆全程3800公里，自驾旅行三年的狗狗这次不带了',
 '带防盗网的土味阳台怎么改造？没关系，我会出手的！',
 '【私人按摩专家有！】定点穴位按摩！在家也能享受到高级理疗店的spa待遇！',
 '失策了，最后中计了…',
 '90后大学老师第一次相亲实录',
 '看小代耕地那么轻松，媳妇也想体验一下，这么累的活还是我自己来',
 '这个东西真的是易胖体质的天敌叭！！',
 '【后背除螨】别在用香皂洗澡了！选对沐浴露告别前胸后背出油长痘！',
 '鉴定网络热门景点视频（28）',
 '回归野人生活后，发现人生的意义就是多吃饭',
 '抬头恶心想吐是脑缺氧前兆？再不抓紧按摩咱就收拾收拾准备预约手术室吧！',
 '越南媳妇说上班有什么感受，老公还体贴的送给她一份礼物',
 '旧房改造丨男生给表哥打造 原木风卧室！智能家居 无主灯',
 '【香薰天花板】车可以不贵，但是一定要有品位！',
 '电动牙刷别交智商税！听我的就够了',
 '我婆婆把我爸折腾废了',
 '当你用过这个之后就不会想再用别的',
 '【好男友养成攻略】别再

In [ ]:
titles=[]
from bilibili_api import video
for i in range(1,len(bvs)):
    try:#print(i)
        bvid=bvs[i]
        video_info=video.get_video_info(bvid=bvid)
        titleyuanben = video_info["title"]
        print(video_info["title"])
        titles.append(titleyuanben)
    except:
        titles.append('')

老 公 试 图 洗 澡 ， 我 开 启 猎 杀 时 刻 ！
老婆突然贤惠，主动下厨，不会是对我有非分之想吧！
结婚，一定要嫁给打游戏的男人，免费代练真香！
毛绒吧唧可以防水吗？纳米涂层让毛绒章疏水防污，耐挤压易清洁
珐琅表面如何防污？纳米涂层让金属徽章吧唧防油性记号笔防粘测试
勋章容易氧化怎么办？纳米涂层有无色透明，密封防潮防腐蚀性能
你们觉得怎么样
体虚无力？别怕这个锌硒片建议男生人手一瓶！助你焕发新生，精神头重返18岁！
拜耳算是把维生素整明白了！别再找借口偷懒不吃维生素了！
幸福感宝藏好物，怎么让所有女生都知道啊！
【人类伟大发明】太顶了！这个冲牙器必须要夸夸！还不赶紧人手一只，从此直接和口臭蛀牙说拜拜！
【理想男友香】闻一次就上瘾的衣物香氛！异味说散就散了！喷上秒成为女生的心动白月光！ 藏进衣服里的伪体香！选对好用的衣物喷雾真的很重要！留香除螨一步搞定！说它比香
【一招远离痛经】女朋友痛经怎么办？快来get新技能
我悟了！女生来姨妈了正确做法居然是.......
别再说智商税了，这玩意儿能不能瘦我们能不知道嘛？！！
油炸！挑战给擦边视频去油…这算成功了吗？？
从上海辞职回家种地，还没开始就花了10000多元，是种什么体验？
亲兄妹的快乐日常，养了一年的小公鸡还不下蛋，含泪带妹吃烤鸡？
叶黄素坚持吃半个月会发生什么
这个吸管勺集吸管、搅拌为一体，比塑料吸管更卫生干净，适合喝花茶，果汁，孕妇和老人喝汤，清洗也简单#吸管勺子
【开箱】入门级手持吸尘器 刚需用户如何选择
VLOG/去见男朋友啦！给他个惊喜，久违的情侣约会日
偷偷把小雨伞放进快递盒，让男友在外面打开！他尴尬到爆炸哈哈哈哈哈
你看我说的吧，年轻人回乡创业很得劲儿！
科普一下：蜂蜜没有办法辨别真假（物理方法）
你管这叫修正带？
VLOG｜全世界都知道我们不是情侣
我对象是意大利楚雨荨！山村男孩靠读书减肥逆转人生！
阿里阿多美羊羊桑
「在京vlog」#该吃吃该喝喝 咱啥事不往心里搁｜国贸dubujib 牛肉嫩豆腐汤｜家庭豪华烤肉局｜吃到广莲申啦｜杜夫朗格 莫比乌斯蛋糕｜超费面包的坚果酱+奶酪
“精神”小伙儿的秘密武器……
清华大四学生毕设竟然是研究小行星防御？！【毕设vlog】
90后抠搜儿媳搭配60后开挂公婆！盖出理想家园！
40度还天天穿吊带短裤没晒黑的的就是我！
太兴奋话也不会说了！
胃癌的我是怎么走

In [3]:
import pandas as pd
df1 = pd.DataFrame()
df1['title']=titles
df1.to_excel(r"C:\Users\CANDY\Desktop\testtitle.xls")

NameError: name 'titles' is not defined

In [5]:
#清除储存文件中原有的截图
def del_file(path):
    for i in os.listdir(path):
        # 如果存在文件夹进行递归
        if os.path.isdir(os.path.join(path, i)):
            del_file(os.path.join(path, i))
        # 如果是文件进行删除
        elif os.path.isfile:
            os.remove(os.path.join(path, i))

In [6]:
#遍历所有截图计算平均HSI
def avghsv(path):
    #初始化
    hs=[]
    ss=[]
    vs=[]
    img_paths=[]
    dir_paths=os.listdir(path)
    #得到每个图像的路径
    for dir_path in dir_paths:
        img_path=os.path.join(path,dir_path)
        img_paths.append(img_path)
    for img in img_paths:
        if img.endswith('.jpg'): 
            #该图片的h,s,v平均值
            pict=cv2.imread(img)
            hsv=cv2.cvtColor(pict,cv2.COLOR_BGR2HSV)#转化成hsv空间
            h=hsv[:,:,0]
            s=hsv[:,:,1]
            v=hsv[:,:,2]#像素矩阵最后一维012对应hsv三个字母
            h_mean=np.mean(h)
            s_mean=np.mean(s)
            v_mean=np.mean(v)
            hs.append(h_mean)
            ss.append(s_mean)
            vs.append(v_mean)
    #求这几个图片的平均hsv
    hs_mean=np.mean(hs)
    ss_mean=np.mean(ss)
    vs_mean=np.mean(vs)
    
    return hs_mean,ss_mean,vs_mean    

In [7]:
#读入视频路径
inputdir=r"D:\CRTubeGet Downloaded"
#输出图片路径
outputdir=r"C:\Users\CANDY\Desktop\video_to_ppt"

In [8]:
import cv2
import os
def process_video(start,end,videodir,outputdir):
    #清除储存文件中原有的截图
    del_file(outputdir)
    # 定义视频文件路径和保存图像文件路径
    video_path =videodir
    save_path =outputdir
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
     # 定义起始和结束时间，以及帧数间隔
    if start==0:
        start_time =start # 单位为秒
    else:
        start_time=start-2
    end_time =end# 单位为秒

    # 打开视频文件
    cap = cv2.VideoCapture(video_path) 
    # 从指定的时间开始读取视频帧
    cap.set(cv2.CAP_PROP_POS_MSEC, start_time *1000) 
    # 定义计数器变量
    i = 0 
    # 读取视频帧，并保存每个间隔的帧为图像文件
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # 判断当前时间是否在指定的时间段内
            current_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            if current_time >= start_time and i<=50:
                print(current_time)
                # 保存当前帧为图像文件
                out_file_name = str(i) + ".jpg"
                out_file_path = os.path.join(save_path, out_file_name)
                cv2.imwrite(out_file_path, frame)
                i += 1
                print("processed frame:", i-1)
            if current_time >= end_time:
                break
        else:
            break
    # 关闭视频文件
    cap.release()
    return i-1

In [9]:
# Hash值对比
def cmpHash(hash1, hash2,shape=(10,10)):
    n = 0
    # hash长度不同则返回-1代表传参出错
    if len(hash1)!=len(hash2):
        return -1
    # 遍历判断
    for i in range(len(hash1)):
        # 相等则n计数+1，n最终为相似度
        if hash1[i] == hash2[i]:
            n = n + 1
    return n/(shape[0]*shape[1])
# 感知哈希算法(pHash)
def pHash(img,shape=(10,10)):
    # 缩放32*32
    img = cv2.resize(img, (32, 32))  # , interpolation=cv2.INTER_CUBIC

    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 将灰度图转为浮点型，再进行dct变换
    dct = cv2.dct(np.float32(gray))
    # opencv实现的掩码操作
    dct_roi = dct[0:10, 0:10]

    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash

In [10]:
for j in range(1,10):
    print(j)

1
2
3
4
5
6
7
8
9


In [11]:
import numpy as np
import jieba
#读取停用词

def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r+',encoding='utf-8').readlines()]
    return stopwords

# 加载停用词
stopwords = stopwordslist(r'C:\Users\CANDY\代码\停用词.txt')

def cosine_similarity(sentence1: str, sentence2: str) -> float:
    """
    :param sentence1: s
    :param sentence2:
    :return: 两句文本的相识度
    """
    seg1 = [word for word in jieba.cut(sentence1) if word not in stopwords]
    seg2 = [word for word in jieba.cut(sentence2) if word not in stopwords]
    word_list = list(set([word for word in seg1 + seg2]))#建立词库
    word_count_vec_1 = []
    word_count_vec_2 = []
    for word in word_list:
        word_count_vec_1.append(seg1.count(word))#文本1统计在词典里出现词的次数
        word_count_vec_2.append(seg2.count(word))#文本2统计在词典里出现词的次数

    vec_1 = np.array(word_count_vec_1)
    vec_2 = np.array(word_count_vec_2)
    #余弦公式

    num = vec_1.dot(vec_2.T)
    denom = np.linalg.norm(vec_1) * np.linalg.norm(vec_2)
    cos = num / denom
    sim = 0.5 + 0.5 * cos

    return sim

In [12]:
#修改路径
import os
import re
import time
  
"""对指定目录下的所有文件进行有选择的修改名称"""
def ReFileName(dirPath,pattern):
    """
    :param dirPath: 文件夹路径
    :param pattern: 正则匹配模式
    :return:
    """
    # 对目录下的文件进行遍历
    #print(os.listdir(dirPath))
    for file in os.listdir(dirPath):
        #print(file)
        # 判断是否是文件
        if os.path.isfile(os.path.join(dirPath, file)) == True:
            # 用正则匹配，去掉不需要的词
            newName = re.sub(pattern, "", file)
            print('newName',newName)
            # 设置新文件名
            newFilename = file.replace(file, newName)
            print('newFilename',newFilename)
            # 重命名
            try:
                os.rename(os.path.join(dirPath, file), os.path.join(dirPath, newFilename))
            except:
                pass
    print("文件名已统一修改成功")  

'''    for file in os.listdir(dirPath):
        # 判断是否是文件
        if os.path.isfile(os.path.join(dirPath, file)) == True:
            # 用正则匹配，去掉不需要的词
            newName = re.sub(pattern, "", file)
            print(newName)
            # 设置新文件名
            newFilename = file.replace(file, newName)
            print(newFilename)
            # 重命名
            os.rename(os.path.join(dirPath, file), os.path.join(dirPath, newFilename))
    print("文件名已统一修改成功")'''
    
if __name__ == '__main__':
    timeStart = time.time()
    Path=r'D:\CRTubeGet Downloaded'
    
#   pattern = re.compile(r'\[{1}(.+)]\.')
    pattern = re.compile(r'-{1}(.+)zh')
    ReFileName(Path,pattern)
    timeEnd = time.time()
    print("程序走了%d秒"%(timeEnd-timeStart))

newName 169_60 清爽舒适！初春服饰购物分享_衣服包包首饰帽子~.srt
newFilename 169_60 清爽舒适！初春服饰购物分享_衣服包包首饰帽子~.srt
newName 169_60 清爽舒适！初春服饰购物分享_衣服包包首饰帽子~.mp4
newFilename 169_60 清爽舒适！初春服饰购物分享_衣服包包首饰帽子~.mp4
newName 169_60 清爽舒适！初春服饰购物分享_衣服包包首饰帽子~.srt
newFilename 169_60 清爽舒适！初春服饰购物分享_衣服包包首饰帽子~.srt
newName 3.8前购物分享｜一起种草好物提前加购～彩妆、女生必备、服饰～.mp4
newFilename 3.8前购物分享｜一起种草好物提前加购～彩妆、女生必备、服饰～.mp4
newName 30岁二胎宝妈医美初体验！肉眼可见变好看了？.srt
newFilename 30岁二胎宝妈医美初体验！肉眼可见变好看了？.srt
newName 30岁二胎宝妈医美初体验！肉眼可见变好看了？.mp4
newFilename 30岁二胎宝妈医美初体验！肉眼可见变好看了？.mp4
newName 30岁二胎宝妈医美初体验！肉眼可见变好看了？.srt
newFilename 30岁二胎宝妈医美初体验！肉眼可见变好看了？.srt
newName 3分钟告别低头纹！在家轻松养成天鹅颈！！.srt
newFilename 3分钟告别低头纹！在家轻松养成天鹅颈！！.srt
newName 3分钟告别低头纹！在家轻松养成天鹅颈！！.mp4
newFilename 3分钟告别低头纹！在家轻松养成天鹅颈！！.mp4
newName 3分钟告别低头纹！在家轻松养成天鹅颈！！.srt
newFilename 3分钟告别低头纹！在家轻松养成天鹅颈！！.srt
newName 3种失眠1个方法！让熬夜失眠人群get深度睡眠！！！.srt
newFilename 3种失眠1个方法！让熬夜失眠人群get深度睡眠！！！.srt
newName 3种失眠1个方法！让熬夜失眠人群get深度睡眠！！！.mp4
newFilename 3种失眠1个方法！让熬夜失眠人群get深度睡眠！！！.mp4
newName 3种失眠1个方法！让熬夜失眠人群get深度睡眠！！！.srt
new

In [13]:
import re,openpyxl
def docsimilarity(path,begin_time,end_time):
    print(begin_time,end_time)
    path=path
    begintime=begin_time
    endtime=end_time
    new = []
    zimubegins=[]
    try:
        
        with open(path, encoding="utf-8-sig") as f:
            for sub in f.readlines():
                if sub[:3]== '00:':
                    begin_min= int(sub.split(':')[1])
                    begin_sec= int(sub.split('-')[0].split(',')[0].split(':')[2])
                    begin=begin_min*60+begin_sec
                    zimubegins.append(begin)
                #print(sub[0],sub[-1])
                #if sub[0] not in list('\n0123456789') and sub[-2] not in list('0123456789'):
                if sub[0] not in list('\n0123456789') and sub[-1] not in list('0123456789'):
                    new.append(sub)
        news = [ele.strip() for ele in new]
        #print(news)
        zimu=pd.DataFrame()
        n=len(news)
        zimubegins=zimubegins[:n]
        zimu['begin']=zimubegins
        zimu['new']=news
        
        ad=''
        noad=''

        for i in range(len(zimu)):
            if begintime<=zimu.begin[i]<=endtime:
                docsimilarity=1
                ad=ad+','+zimu.new[i]
            else:
                noad=noad+','+zimu.new[i]
        #print('ad',ad)
        #print('noad',noad)
        if len(noad)==0 and len(ad)!=0:
            docsimilarity=1
        elif len(ad)==0 and len(noad)!=0:
            docsimilarity=0
        else:
            docsimilarity=cosine_similarity(ad,noad)  
    except:
        pass

    return docsimilarity    

In [52]:
file_list=os.listdir(inputdir)
error=[]
video=pd.DataFrame()
#文件夹下有多少视频
for j in range(92,93):
    print('###')
    titleyuanben=titleyuanbens[j]
    start_time=int(begins[j])
    end_time=int(ends[j])
    print(titleyuanben)
    title=titleyuanben
    h=0
    v=0
    s=0
    similarity=0
    for i in range(len(file_list)):
        inputdir_title=file_list[i].replace('.mp4','')
        #title=inputdir_title.replace('.mp4','')
        if inputdir_title==titleyuanben:
            try:
                print("Step 1:对视频截帧")
                videodir=os.path.join(inputdir,inputdir_title)+'.mp4'
                print(videodir)
                n=process_video(start_time,end_time,videodir,outputdir) 
                print(n)

                print("Step 2:")
                print("计算图片相似度")
                if end_time==0:
                    similarity=1.0
                else:
                    img1 = cv2.imread(r"C:\Users\CANDY\Desktop\video_to_ppt\0.jpg")  
                    img2 = cv2.imread(r"C:\Users\CANDY\Desktop\video_to_ppt\%d.jpg"%(n))
                    hash1 = pHash(img1)
                    hash2 = pHash(img2)
                    similarity=cmpHash(hash1, hash2)
                print(similarity)
                print("Step 3:")
                print("计算广告平均HSV")
                h,s,v=avghsv(outputdir)
                print('h平均值为%s,\ns平均值为%s,\nv平均值为%s'%(h,s,v))

                #item={'title':title,'h':h,'s':s,'v':v,'similarity':similarity}
                #video=video.append(item,ignore_index=True)
            except:
                inputdir_title=file_list[i]
                error.append(inputdir_title)
    item={'title':title,'h':h,'s':s,'v':v,'picsimilarity':similarity}
    video=video.append(item,ignore_index=True)

###
那些胸大辣妹的秘密_一件内衣就能无中生有
Step 1:对视频截帧
D:\CRTubeGet Downloaded\那些胸大辣妹的秘密_一件内衣就能无中生有.mp4
58.03
processed frame: 0
58.064
processed frame: 1
58.098
processed frame: 2
58.132
processed frame: 3
58.165
processed frame: 4
58.199
processed frame: 5
58.233000000000004
processed frame: 6
58.267
processed frame: 7
58.301
processed frame: 8
58.334
processed frame: 9
58.368
processed frame: 10
58.402
processed frame: 11
58.436
processed frame: 12
58.47
processed frame: 13
58.503
processed frame: 14
58.537
processed frame: 15
58.571
processed frame: 16
58.605000000000004
processed frame: 17
58.639
processed frame: 18
58.672000000000004
processed frame: 19
58.706
processed frame: 20
58.74
processed frame: 21
58.774
processed frame: 22
58.808
processed frame: 23
58.841
processed frame: 24
58.875
processed frame: 25
58.909
processed frame: 26
58.943
processed frame: 27
58.977000000000004
processed frame: 28
59.01
processed frame: 29
59.044000000000004
processed frame: 30
59.078
processed frame: 31
59.

In [50]:
video

,title,h,s,v,picsimilarity
0,大胸女孩的显瘦神器_一款内衣由内而外改变气质,65.765722,75.072445,140.939244,0.64


In [51]:
#文字相似度
file_list=os.listdir(inputdir)
video1=pd.DataFrame()
#文件夹下有多少视频
for j in range(92,93):
    print('###')
    titleyuanben=titleyuanbens[j]
    start_time=int(begins[j])
    end_time=int(ends[j])
    print(titleyuanben)
    title=titleyuanben
    similarity=111
    for i in range(len(file_list)):
        inputdir_title=file_list[i].replace('.srt','')
        
        #title=inputdir_title.replace('.mp4','')
        if inputdir_title==titleyuanben:
            if end_time==0:
                similarity=0
            else:
                print('计算文本相似度')    
                srtdir=os.path.join(inputdir,inputdir_title)+'.srt'
                print(srtdir)
                similarity=docsimilarity(srtdir,begins[j],ends[j])
                print(similarity)
    item={'title':titleyuanben,'docsimilarity':similarity}
    video1=video1.append(item,ignore_index=True)

###
那些胸大辣妹的秘密_一件内衣就能无中生有
计算文本相似度
D:\CRTubeGet Downloaded\那些胸大辣妹的秘密_一件内衣就能无中生有.srt
60.0 173.0
0.6805263778401988


In [42]:
video.to_csv(r'C:\Users\CANDY\Desktop\video.csv',encoding="utf_8_sig")

In [43]:
video1.to_csv(r'C:\Users\CANDY\Desktop\video1.csv',encoding="utf_8_sig")